In [167]:
import pandas as pd
import numpy as np

In [168]:
train_data=pd.read_csv("twitter_train.csv")
test_data=pd.read_csv("twitter_test.csv")
train_data.head()

,tweet_id,airline_sentiment,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,567900433542488064,negative,Southwest,NaN,ColeyGirouard,NaN,0,"@SouthwestAir I am scheduled for the morning, ...",NaN,2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,569989168903819264,positive,Southwest,NaN,WalterFaddoul,NaN,0,@SouthwestAir seeing your workers time in and ...,NaN,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,568089179520954368,positive,United,NaN,LocalKyle,NaN,0,@united Flew ORD to Miami and back and had gr...,NaN,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,568928195581513728,negative,Southwest,NaN,amccarthy19,NaN,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,NaN,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,568594180014014464,negative,United,NaN,J_Okayy,NaN,0,@united so our flight into ORD was delayed bec...,NaN,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [100]:
xtrain=train_data["text"]
ytrain=train_data["airline_sentiment"]
xtest=test_data["text"]

In [170]:
neg=0
neu=0
pos=0
for sent in ytrain:
    if sent=="negative":
        neg+=1
    elif sent=="neutral":
        neu+=1
    else:
        pos+=1
neg,neu,pos

(6851, 2327, 1802)

In [171]:
from nltk.corpus import stopwords
stop=set(stopwords.words("english"))
import string
punctuations=list(string.punctuation)
stop.update(punctuations)

In [172]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet
Lemmatizer=WordNetLemmatizer()
#pos_tag(["better"])
#Lemmatizer.lemmatize("better",'a')

In [173]:
def simple_pos(tag):
    if tag.startswith("N"):
        return wordnet.NOUN
    elif tag.startswith("V"):
        return wordnet.VERB
    elif tag.startswith("J"):
        return wordnet.ADJ
    elif tag.startswith("R"):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [174]:
xtrain[0]

'@SouthwestAir I am scheduled for the morning, 2 days after the fact, yes..not sure why my evening flight was the only one Cancelled Flightled'

In [175]:
from nltk import word_tokenize
x_train=[]
for text in xtrain:
    x_train.append(word_tokenize(text))
x_test=[]
for text in xtest:
    x_test.append(word_tokenize(text))

In [176]:
x_train[0]

['@',
 'SouthwestAir',
 'I',
 'am',
 'scheduled',
 'for',
 'the',
 'morning',
 ',',
 '2',
 'days',
 'after',
 'the',
 'fact',
 ',',
 'yes..not',
 'sure',
 'why',
 'my',
 'evening',
 'flight',
 'was',
 'the',
 'only',
 'one',
 'Cancelled',
 'Flightled']

In [177]:
def clean(text_data):
    X_Train=[]
    for text in text_data:
        clean_words=[]
        for word in text:
            if word.lower() not in stop:
                tag=pos_tag([word])[0][1]
                clean_word=Lemmatizer.lemmatize(word,simple_pos(tag))
                clean_words.append(clean_word)
        X_Train.append(clean_words)
    return X_Train

In [178]:
X_Train=clean(x_train)
X_Test=clean(x_test)

In [179]:
X_Train[0]

['SouthwestAir',
 'schedule',
 'morning',
 '2',
 'day',
 'fact',
 'yes..not',
 'sure',
 'even',
 'flight',
 'one',
 'Cancelled',
 'Flightled']

In [180]:
xTrain=[" ".join(text) for text in X_Train]
xTest=[" ".join(text) for text in X_Test]

In [181]:
from sklearn.feature_extraction.text import CountVectorizer
count_vec=CountVectorizer(max_features=3000,ngram_range=(1,3))
Xtrain=count_vec.fit_transform(xTrain)
Xtest=count_vec.transform(xTest)

In [182]:
from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB()
clf.fit(Xtrain,ytrain)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [183]:
clf.score(Xtrain,ytrain)

0.8098360655737705

In [184]:
Ypred=clf.predict(Xtest)

In [185]:
np.savetxt("pred.csv",Ypred,delimiter=",",fmt="%s")